# Discrete Helmholtz Hodge Decomposition and Applications

With this notebook, you can experiment with the discrete Helmholtz Hodge decomposition for nullspace consistent classical finite difference summation by parts operators as described in the paper.

Run the cell below to activate the project environment containing the necessary packages.

In [ ]:
]activate .

# Setup

Run this cell once to setup some basic functions.

In [ ]:
using SummationByPartsOperators
using LinearAlgebra, SparseArrays
using IterativeSolvers
using Printf
using PyPlot, LaTeXStrings; plt = PyPlot
using PyCall

# line cycler adapted to colourblind people
cycler = pyimport("cycler")
cycler.cycler
colours = ["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442"]
linestyles = ["-", "--", "-.", ":", "-", "--", "-."]
#markers = [">", "^", "<", "v", "+", "x", "."]
markers = ["4", "2", "3", "1", "+", "x", "."]
colourblind_cycler = (cycler.cycler(color=colours) + cycler.cycler(linestyle=linestyles))
plt.rc("axes", prop_cycle=colourblind_cycler)
plt.rc("lines", linewidth=2, markersize=12, markeredgewidth=1.5)

plt.rc("text", usetex=true)
plt.rc("text.latex", preamble="\\usepackage{newpxtext}\\usepackage{newpxmath}\\usepackage{commath}\\usepackage{mathtools}")
plt.rc("font", family="serif", size=18.)
plt.rc("savefig", dpi=200)
plt.rc("legend", fontsize="medium", fancybox=true, framealpha=0.5)

isdir("../figures") || mkdir("../figures")

function matrices(N, order)
    d_op = derivative_operator(MattssonNordström2004(), 1, order, -1., 1., N)
    x = SummationByPartsOperators.grid(d_op)
    d = sparse(d_op)
    m = mass_matrix(d_op)
    id = one(d)
    
    x, d, m, id
end

function operators(Nx, Ny, order)
    x, dx, mx, idx = matrices(Nx, order)
    y, dy, my, idy = matrices(Ny, order)
    
    xone = fill(1, size(x))
    yone = fill(1, size(y))

    x1 = kron(x, yone)
    x2 = kron(xone, y)

    Dx = kron(dx, idy)
    Dy = kron(idx, dy)
    M = kron(mx, my)
    Mvec = Diagonal(vcat(M.diag, M.diag))

    grad = vcat(Dx, Dy)
    rot = vcat(-Dy, Dx)
    curl = hcat(-Dy, Dx) 
    div = hcat(Dx, Dy) 
    
    x1, x2, grad, rot, curl, div, M, Mvec
end

function operators(Nx, Ny, Nz, order)
    x, dx, mx, idx = matrices(Nx, order)
    y, dy, my, idy = matrices(Ny, order)
    z, dz, mz, idz = matrices(Nz, order)
    
    xone = fill(1, size(x))
    yone = fill(1, size(y))
    zone = fill(1, size(z))

    x1 = kron(x, yone, zone)
    x2 = kron(xone, y, zone)
    x3 = kron(xone, yone, z)

    Dx = kron(dx, idy, idz)
    Dy = kron(idx, dy, idz)
    Dz = kron(idx, idy, dz)
    M = kron(mx, my, mz)
    Mvec = Diagonal(vcat(M.diag, M.diag, M.diag))

    grad = vcat(Dx, Dy, Dz) 
    curl = [zero(Dx) -Dz Dy; Dz zero(Dx) -Dx; -Dy Dx zero(Dx)] 
    div = hcat(Dx, Dy, Dz) 
    
    x1, x2, x3, grad, curl, div, M, Mvec
end


print_key_value(retval, key) = @printf("%-33s = %9.2e \n", key, retval[key])


function compute_potentials(u1func, u2func, N1, N2, order; 
                            scalar_potential_first=true,
                            φ_analytical=nothing,
                            v_analytical=nothing,
                            u1_solenoidal=nothing,
                            u2_solenoidal=nothing,
                            u1_irrotational=nothing,
                            u2_irrotational=nothing,
                            atol=1.e-12, btol=1.e-12, verbose=true, save_fields=true)
    x1, x2, grad, rot, curl, div, M, Mvec = operators(N1, N2, order)
    sqrtM = sqrt(M)
    sqrtMvec = sqrt(Mvec)

    u1 = u1func.(x1, x2)
    u2 = u2func.(x1, x2)
    u = vcat(u1, u2)
    
    retval = Dict{String, Any}()
    retval["||u||_M"] = sqrt(u' * M * u)
    
    if scalar_potential_first
        # solve for the scalar potential
        if verbose
            println("Computing scalar potential...")
            @time φ = sqrtM \ lsqr(sqrtMvec*grad/sqrtM, sqrtMvec*u, atol=atol, btol=btol)
        else
            φ = sqrtM \ lsqr(sqrtMvec*grad/sqrtM, sqrtMvec*u, atol=atol, btol=btol)
        end
        gradφ = grad * φ

        r = u - gradφ
        retval["< u - grad φ, grad φ >_M"] = r' * M * gradφ
        retval["||u - grad φ||_M"] = sqrt(r' * M * r)

        # solve for the vector potential
        if verbose
            println("Computing vector potential...")
            @time v = sqrtM \ lsqr(sqrtMvec*rot/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        else
            v = sqrtM \ lsqr(sqrtMvec*rot/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        end
        rotv = rot * v

        @. r = r - rotv
        retval["< u - grad φ - rot v, rot v >_M"] = r' * M * rotv
        retval["||u - grad φ - rot v||_M"] = sqrt(r' * M * r)
        retval["||r||_M"] = retval["||u - grad φ - rot v||_M"]
    else
        # solve for the vector potential
        if verbose
            println("Computing vector potential...")
            @time v = sqrtM \ lsqr(sqrtMvec*rot/sqrtM, sqrtMvec*u, atol=atol, btol=btol)
        else
            v = sqrtM \ lsqr(sqrtMvec*rot/sqrtM, sqrtMvec*u, atol=atol, btol=btol)
        end
        rotv = rot * v

        r = u - rotv
        retval["< u - rot v, rot v >_M"] = r' * M * rotv
        retval["||u - rot v||_M"] = sqrt(r' * M * r)
        
        # solve for the scalar potential
        if verbose
            println("Computing scalar potential...")
            @time φ = sqrtM \ lsqr(sqrtMvec*grad/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        else
            φ = sqrtM \ lsqr(sqrtMvec*grad/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        end
        gradφ = grad * φ

        @. r = r - gradφ
        retval["< u - rot v - grad φ, grad φ >_M"] = r' * M * gradφ
        retval["||u - rot v - grad φ||_M"] = sqrt(r' * M * r)
        retval["||r||_M"] = retval["||u - rot v - grad φ||_M"]
    end
    
    if save_fields
        retval["u"] = u
        retval["grad φ"] = gradφ
        retval["rot v"] = rotv
        retval["r"] = r
        retval["x1"] = x1
        retval["x2"] = x2
    end
    
    if φ_analytical !== nothing
        φ_ana = φ_analytical.(x1, x2)
        diff = φ - φ_ana
        retval["||φ - φ_ana||_M"] = sqrt(diff' * M * diff)
    end
    
    if v_analytical !== nothing
        v_ana = v_analytical.(x1, x2)
        diff = v - v_ana
        retval["||v - v_ana||_M"] = sqrt(diff' * M * diff)
    end
    
    if u1_solenoidal !== nothing && u2_solenoidal !== nothing
        u1_sol = u1_solenoidal.(x1, x2)
        u2_sol = u2_solenoidal.(x1, x2)
        u_sol = vcat(u1_sol, u2_sol)
        diff = rotv - u_sol
        retval["||rot v - u_sol||_M"] = sqrt(diff' * Mvec * diff)
    end
    
    if u1_irrotational !== nothing && u2_irrotational !== nothing
        u1_irr = u1_irrotational.(x1, x2)
        u2_irr = u2_irrotational.(x1, x2)
        u_irr = vcat(u1_irr, u2_irr)
        diff = gradφ - u_irr
        retval["||grad φ - u_irr||_M"] = sqrt(diff' * Mvec * diff)
    end
    
    retval
end

function compute_potentials(u1func, u2func, u3func, N1, N2, N3, order; 
                            scalar_potential_first=true,
                            φ_analytical=nothing,
                            v1_analytical=nothing,
                            v2_analytical=nothing,
                            v3_analytical=nothing,
                            u1_solenoidal=nothing,
                            u2_solenoidal=nothing,
                            u3_solenoidal=nothing,
                            u1_irrotational=nothing,
                            u2_irrotational=nothing,
                            u3_irrotational=nothing,
                            atol=1.e-12, btol=1.e-12, verbose=true, linsolve=lsmr, save_fields=true)
    x1, x2, x3, grad, curl, div, M, Mvec = operators(N1, N2, N3, order)
    sqrtM = sqrt(M)
    sqrtMvec = sqrt(Mvec)

    u1 = u1func.(x1, x2, x3)
    u2 = u2func.(x1, x2, x3)
    u3 = u3func.(x1, x2, x3)
    u = vcat(u1, u2, u3)
    
    retval = Dict{String, Any}()
    retval["||u||_M"] = sqrt(u' * M * u)
    
    if scalar_potential_first
        # solve for the scalar potential
        if verbose
            println("Computing scalar potential...")
            @time φ = sqrtM \ linsolve(sqrtMvec*grad/sqrtM, sqrtMvec*u, atol=atol, btol=btol)
        else
            φ = sqrtM \ linsolve(sqrtMvec*grad/sqrtM, sqrtMvec*u, atol=atol, btol=btol)
        end
        gradφ = grad * φ

        r = u - gradφ
        retval["< u - grad φ, grad φ >_M"] = r' * M * gradφ
        retval["||u - grad φ||_M"] = sqrt(r' * M * r)

        # solve for the vector potential
        if verbose
            println("Computing vector potential...")
            @time v = sqrtMvec \ linsolve(sqrtMvec*curl/sqrtMvec, sqrtMvec*r, atol=atol, btol=btol)
        else
            v = sqrtMvec \ linsolve(sqrtMvec*curl/sqrtMvec, sqrtMvec*r, atol=atol, btol=btol)
        end
        curlv = curl * v

        @. r = r - curlv
        retval["< u - grad φ - curl v, curl v >_M"] = r' * M * curlv
        retval["||u - grad φ - curl v||_M"] = sqrt(r' * M * r)
        retval["||r||_M"] = retval["||u - grad φ - curl v||_M"]
    else
        # solve for the vector potential
        if verbose
            println("Computing vector potential...")
            @time v = sqrtMvec \ linsolve(sqrtMvec*curl/sqrtMvec, sqrtMvec*u, atol=atol, btol=btol)
        else
            v = sqrtMvec \ linsolve(sqrtMvec*curl/sqrtMvec, sqrtMvec*u, atol=atol, btol=btol)
        end
        curlv = curl * v

        r = u - curlv
        retval["< u - curl v, curl v >_M"] = r' * M * curlv
        retval["||u - curl v||_M"] = sqrt(r' * M * r)
        
        # solve for the scalar potential
        if verbose
            println("Computing scalar potential...")
            @time φ = sqrtM \ linsolve(sqrtMvec*grad/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        else
            φ = sqrtM \ linsolve(sqrtMvec*grad/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        end
        gradφ = grad * φ

        @. r = r - gradφ
        retval["< u - curl v - grad φ, grad φ >_M"] = r' * M * gradφ
        retval["||u - curl v - grad φ||_M"] = sqrt(r' * M * r)
        retval["||r||_M"] = retval["||u - curl v - grad φ||_M"]
    end
    
    if save_fields
        retval["u"] = u
        retval["grad φ"] = gradφ
        retval["curl v"] = curlv
        retval["r"] = r
        retval["x1"] = x1
        retval["x2"] = x2
        retval["x3"] = x3
    end
    
    if φ_analytical !== nothing
        φ_ana = φ_analytical.(x1, x2, x3)
        diff = φ - φ_ana
        retval["||φ - φ_ana||_M"] = sqrt(diff' * M * diff)
    end
    
    if v1_analytical !== nothing && v2_analytical !== nothing && v3_analytical !== nothing
        v1_ana = v1_analytical.(x1, x2, x3)
        v2_ana = v2_analytical.(x1, x2, x3)
        v3_ana = v3_analytical.(x1, x2, x3)
        v_ana = vcat(v1_ana, v2_ana, v3_ana)
        diff = v - v_ana
        retval["||v - v_ana||_M"] = sqrt(diff' * Mvec * diff)
    end
    
    if u1_solenoidal !== nothing && u2_solenoidal !== nothing && u3_solenoidal !== nothing
        u1_sol = u1_solenoidal.(x1, x2, x3)
        u2_sol = u2_solenoidal.(x1, x2, x3)
        u3_sol = u3_solenoidal.(x1, x2, x3)
        u_sol = vcat(u1_sol, u2_sol, u3_sol)
        diff = curlv - u_sol
        retval["||curl v - u_sol||_M"] = sqrt(diff' * Mvec * diff)
    end
    
    if u1_irrotational !== nothing && u2_irrotational !== nothing && u3_irrotational !== nothing
        u1_irr = u1_irrotational.(x1, x2, x3)
        u2_irr = u2_irrotational.(x1, x2, x3)
        u3_irr = u3_irrotational.(x1, x2, x3)
        u_irr = vcat(u1_irr, u2_irr, u3_irr)
        diff = gradφ - u_irr
        retval["||grad φ - u_irr||_M"] = sqrt(diff' * Mvec * diff)
    end
    
    retval
end


function convergence_test(u1func, u2func, Ns, order,
                          scalar_potential_first,
                          φ_analytical, v_analytical,
                          u1_solenoidal, u2_solenoidal,
                          u1_irrotational, u2_irrotational;
                          atol=1.e-12, btol=1.e-12, verbose=true)
    
    errors = Dict{String, Any}()
    errors["||φ - φ_ana||_M"] = fill(NaN, length(Ns))
    errors["||v - v_ana||_M"] = fill(NaN, length(Ns))
    errors["||rot v - u_sol||_M"] = fill(NaN, length(Ns))
    errors["||grad φ - u_irr||_M"] = fill(NaN, length(Ns))
    errors["||r||_M"] = fill(NaN, length(Ns))
    error_keys = deepcopy(keys(errors))
    errors["N"] = Ns
    
    for (idx,N) in enumerate(Ns)
        N1 = N2 = N
        retval = compute_potentials(u1func, u2func, N1, N2, order,
                                    scalar_potential_first=scalar_potential_first,
                                    φ_analytical=φ_analytical, v_analytical=v_analytical,
                                    u1_solenoidal=u1_solenoidal, u2_solenoidal=u2_solenoidal,
                                    u1_irrotational=u1_irrotational, u2_irrotational=u2_irrotational,
                                    verbose=verbose, save_fields=false)
        for key in error_keys
            errors[key][idx] = retval[key]
        end
    end
    
    errors
end

function convergence_test(u1func, u2func, u3func, Ns, order,
                          scalar_potential_first,
                          φ_analytical, 
                          v1_analytical, v2_analytical, v3_analytical,
                          u1_solenoidal, u2_solenoidal, u3_solenoidal,
                          u1_irrotational, u2_irrotational, u3_irrotational;
                          atol=1.e-12, btol=1.e-12, verbose=true)
    
    errors = Dict{String, Any}()
    errors["||φ - φ_ana||_M"] = fill(NaN, length(Ns))
    errors["||v - v_ana||_M"] = fill(NaN, length(Ns))
    errors["||curl v - u_sol||_M"] = fill(NaN, length(Ns))
    errors["||grad φ - u_irr||_M"] = fill(NaN, length(Ns))
    errors["||r||_M"] = fill(NaN, length(Ns))
    error_keys = deepcopy(keys(errors))
    errors["N"] = Ns
    
    for (idx,N) in enumerate(Ns)
        N1 = N2 = N3 = N
        retval = compute_potentials(u1func, u2func, u3func, N1, N2, N3, order,
                                    scalar_potential_first=scalar_potential_first,
                                    φ_analytical=φ_analytical, 
                                    v1_analytical=v1_analytical, v2_analytical=v2_analytical, v3_analytical=v3_analytical,
                                    u1_solenoidal=u1_solenoidal, u2_solenoidal=u2_solenoidal, u3_solenoidal=u3_solenoidal,
                                    u1_irrotational=u1_irrotational, u2_irrotational=u2_irrotational, u3_irrotational=u3_irrotational,
                                    verbose=verbose, linsolve=lsmr)
        for key in error_keys
            errors[key][idx] = retval[key]
        end
    end
    
    errors
end


function convergence_orders(Ns, errors)
    @assert length(Ns) == length(errors)
    
    orders = zero(errors)
    orders[1] = NaN
    for i in 2:length(errors)
        orders[i] = - log(errors[i-1] / errors[i]) / log(Ns[i-1] / Ns[i])
    end
    orders
end


# Visualise Remaining Term $r$: Grid Oscillations in 2D

In [ ]:
# Ahusborde et al. (2007)
u1func = (x1,x2) -> -sinpi(x1)*cospi(x2) + π*cospi(x1+x2)
u2func = (x1,x2) -> cospi(x1)*sinpi(x2) + π*cospi(x1+x2)
φ_analytical = (x1,x2) -> sinpi(x1+x2)
v_analytical = (x1,x2) -> sinpi(x1)*sinpi(x2)/π
u1_solenoidal = (x1,x2) -> -sinpi(x1)*cospi(x2)
u2_solenoidal = (x1,x2) -> cospi(x1)*sinpi(x2)
u1_irrotational = (x1,x2) -> π*cospi(x1+x2)
u2_irrotational = (x1,x2) -> π*cospi(x1+x2)

N1 = N2 = 60
order = 6
scalar_potential_first = true

retval = compute_potentials(u1func, u2func, N1, N2, order,
                            scalar_potential_first=scalar_potential_first,
                            φ_analytical=φ_analytical, v_analytical=v_analytical,
                            u1_solenoidal=u1_solenoidal, u2_solenoidal=u2_solenoidal,
                            u1_irrotational=u1_irrotational, u2_irrotational=u2_irrotational,
                            verbose=true)

for key in keys(retval)
    isa(retval[key], Number) || continue
    print_key_value(retval, key)
end
sleep(0.1)

close("all")
clf() 

rmin, rmax = extrema(retval["r"])
pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
    reshape(retval["r"][1:N1*N2], N2, N1),
    vmin=rmin, vmax=rmax, linewidth=0, rasterized=true)
xlabel(L"x")
ylabel(L"y")
colorbar()
savefig("../figures/remainder_r1.pdf", bbox_inches="tight")

clf()
pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
    reshape(retval["r"][N1*N2+1:2*N1*N2], N2, N1),
    vmin=rmin, vmax=rmax, linewidth=0, rasterized=true)
xlabel(L"x")
ylabel(L"y")
colorbar()
savefig("../figures/remainder_r2.pdf", bbox_inches="tight")

# Convergence Test 2D

In [ ]:
# Ahusborde et al. (2007)
u1func = (x1,x2) -> -sinpi(x1)*cospi(x2) + π*cospi(x1+x2)
u2func = (x1,x2) -> cospi(x1)*sinpi(x2) + π*cospi(x1+x2)
φ_analytical = (x1,x2) -> sinpi(x1+x2)
v_analytical = (x1,x2) -> sinpi(x1)*sinpi(x2)/π
u1_solenoidal = (x1,x2) -> -sinpi(x1)*cospi(x2)
u2_solenoidal = (x1,x2) -> cospi(x1)*sinpi(x2)
u1_irrotational = (x1,x2) -> π*cospi(x1+x2)
u2_irrotational = (x1,x2) -> π*cospi(x1+x2)

Ns = 41:20:141
scalar_potential_first = true

for order in 2:2:8

    errors = convergence_test(u1func, u2func, Ns, order,
                              scalar_potential_first,
                              φ_analytical, v_analytical,
                              u1_solenoidal, u2_solenoidal,
                              u1_irrotational, u2_irrotational,
                              verbose=false)

    close("all")
    clf()

    println(convergence_orders(errors["N"], errors["||φ - φ_ana||_M"]))
    plot(errors["N"], errors["||φ - φ_ana||_M"], label=L"||\varphi - \varphi_{\mathrm{ana}}||_M", 
        marker=markers[1])

    println(convergence_orders(errors["N"], errors["||v - v_ana||_M"]))
    plot(errors["N"], errors["||v - v_ana||_M"], label=L"||v - v_{\mathrm{ana}}||_M", 
        marker=markers[2], ms=12, mew=1.5, lw=2)

    println(convergence_orders(errors["N"], errors["||grad φ - u_irr||_M"]))
    plot(errors["N"], errors["||grad φ - u_irr||_M"], label=L"||\mathrm{grad}\, \varphi - u_{\mathrm{irr}}||_M", 
        marker=markers[3], ms=12, mew=1.5, lw=2)

    println(convergence_orders(errors["N"], errors["||rot v - u_sol||_M"]))
    plot(errors["N"], errors["||rot v - u_sol||_M"], label=L"||\mathrm{rot}\, v - u_{\mathrm{sol}}||_M", 
        marker=markers[4], ms=12, mew=1.5, lw=2)

    println(convergence_orders(errors["N"], errors["||r||_M"]))
    plot(errors["N"], errors["||r||_M"], label=L"||r||_M", 
        marker=markers[5], ms=12, mew=1.5, lw=2)
    
    order_color = "silver"
    if order == 2
        plot([41, 141], [1.e-3, 1.e-3*(41/141)^2], "-", color=order_color)
        annotate("Order 2", (41, 4.5e-4), color=order_color)
    elseif order == 4
        plot([41, 141], [3.e-4, 3.e-4*(41/141)^3], "-", color=order_color)
        annotate("Order 3", (41, 1.e-4), color=order_color)
    elseif order == 6
        plot([41, 141], [7.e-5, 7.e-5*(41/141)^4.6], "-", color=order_color)
        annotate("Order 4.6", (41, 1.e-5), color=order_color)
        plot([41, 141], [2.5e-3, 2.5e-3*(41/141)^4], "-", color=order_color)
        annotate("Order 4", (45, 2.e-3), color=order_color)
    elseif order == 8
        plot([41, 141], [2.e-5, 2.e-5*(41/141)^5], "-", color=order_color)
        annotate("Order 5", (41, 3.5e-6), color=order_color)
    end

    xlabel(L"N")
    xscale("log")
    ylabel("Error")
    yscale("log")
    savefig("../figures/convergence_2D__order_$order.pdf", bbox_inches="tight", dpi=100)
    
    # create a legend
    order == 2 || continue
    ax = gca()
    handles, labels = ax.get_legend_handles_labels()
    figure()
    figlegend(handles, labels, loc="center", ncol=5)
    savefig("../figures/convergence_2D__legend.pdf", bbox_inches="tight")
end


# Experiments in 3D (Not Used in the Paper)

This cell provides the possibility to experiment a bit with different data and paramters.

In [ ]:
#= curl free vector field
u1func = (x1,x2,x3) -> sinpi(x1) * cospi(x2) * cospi(x3)
u2func = (x1,x2,x3) -> cospi(x1) * sinpi(x2) * cospi(x3)
u3func = (x1,x2,x3) -> cospi(x1) * cospi(x2) * sinpi(x3)
φ_analytical = (x1,x2,x3) -> -cospi(x1) * cospi(x2) * cospi(x3) / π
v1_analytical = (x1,x2,x3) -> zero(x1+x2+x3)
v2_analytical = (x1,x2,x3) -> zero(x1+x2+x3)
v3_analytical = (x1,x2,x3) -> zero(x1+x2+x3)
u1_solenoidal = (x1,x2,x3) -> zero(x1+x2+x3)
u2_solenoidal = (x1,x2,x3) -> zero(x1+x2+x3)
u3_solenoidal = (x1,x2,x3) -> zero(x1+x2+x3)
u1_irrotational = (x1,x2,x3) -> sinpi(x1) * cospi(x2) * cospi(x3)
u2_irrotational = (x1,x2,x3) -> cospi(x1) * sinpi(x2) * cospi(x3)
u3_irrotational = (x1,x2,x3) -> cospi(x1) * cospi(x2) * sinpi(x3)
=#
# new test case
φ_analytical = (x1,x2,x3) -> sinpi(x1) * sinpi(x2) * sinpi(x3) / π
v1_analytical = (x1,x2,x3) -> sinpi(x1) * cospi(x2) * cospi(x3) / π
v2_analytical = (x1,x2,x3) -> cospi(x1) * sinpi(x2) * cospi(x3) / π
v3_analytical = (x1,x2,x3) -> -2 * cospi(x1) * cospi(x2) * sinpi(x3) / π
u1_solenoidal = (x1,x2,x3) -> 3 * cospi(x1) * sinpi(x2) * sinpi(x3)
u2_solenoidal = (x1,x2,x3) -> -3 * sinpi(x1) * cospi(x2) * sinpi(x3)
u3_solenoidal = (x1,x2,x3) -> zero(x1+x2+x3)
u1_irrotational = (x1,x2,x3) -> cospi(x1) * sinpi(x2) * sinpi(x3)
u2_irrotational = (x1,x2,x3) -> sinpi(x1) * cospi(x2) * sinpi(x3)
u3_irrotational = (x1,x2,x3) -> sinpi(x1) * sinpi(x2) * cospi(x3)
u1func = (x1,x2,x3) -> u1_solenoidal(x1,x2,x3) + u1_irrotational(x1,x2,x3)
u2func = (x1,x2,x3) -> u2_solenoidal(x1,x2,x3) + u2_irrotational(x1,x2,x3)
u3func = (x1,x2,x3) -> u3_solenoidal(x1,x2,x3) + u3_irrotational(x1,x2,x3)


N1 = N2 = N3 = 41
order = 2
scalar_potential_first = false

retval = compute_potentials(u1func, u2func, u3func, N1, N2, N3, order,
                            scalar_potential_first=scalar_potential_first,
                            φ_analytical=φ_analytical,
                            v1_analytical=v1_analytical, v2_analytical=v2_analytical, v3_analytical=v3_analytical,
                            u1_solenoidal=u1_solenoidal, u2_solenoidal=u2_solenoidal, u3_solenoidal=u3_solenoidal,
                            u1_irrotational=u1_irrotational, u2_irrotational=u2_irrotational, u3_irrotational=u3_irrotational,
                            verbose=true, linsolve=lsmr)

for key in keys(retval)
    isa(retval[key], Number) || continue
    print_key_value(retval, key)
end
sleep(0.1)


# Convergence Test 3D

In [ ]:
# new test case
φ_analytical = (x1,x2,x3) -> sinpi(x1) * sinpi(x2) * sinpi(x3) / π
v1_analytical = (x1,x2,x3) -> sinpi(x1) * cospi(x2) * cospi(x3) / π
v2_analytical = (x1,x2,x3) -> cospi(x1) * sinpi(x2) * cospi(x3) / π
v3_analytical = (x1,x2,x3) -> -2 * cospi(x1) * cospi(x2) * sinpi(x3) / π
u1_solenoidal = (x1,x2,x3) -> 3 * cospi(x1) * sinpi(x2) * sinpi(x3)
u2_solenoidal = (x1,x2,x3) -> -3 * sinpi(x1) * cospi(x2) * sinpi(x3)
u3_solenoidal = (x1,x2,x3) -> zero(x1+x2+x3)
u1_irrotational = (x1,x2,x3) -> cospi(x1) * sinpi(x2) * sinpi(x3)
u2_irrotational = (x1,x2,x3) -> sinpi(x1) * cospi(x2) * sinpi(x3)
u3_irrotational = (x1,x2,x3) -> sinpi(x1) * sinpi(x2) * cospi(x3)
u1func = (x1,x2,x3) -> u1_solenoidal(x1,x2,x3) + u1_irrotational(x1,x2,x3)
u2func = (x1,x2,x3) -> u2_solenoidal(x1,x2,x3) + u2_irrotational(x1,x2,x3)
u3func = (x1,x2,x3) -> u3_solenoidal(x1,x2,x3) + u3_irrotational(x1,x2,x3)

Ns = 41:20:121
scalar_potential_first = false

for order in 2:2:8
    errors = convergence_test(u1func, u2func, u3func, Ns, order,
                              scalar_potential_first,
                              φ_analytical, 
                              v1_analytical, v2_analytical, v3_analytical,
                              u1_solenoidal, u2_solenoidal, u3_solenoidal,
                              u1_irrotational, u2_irrotational, u3_irrotational,
                              verbose=true)

    close("all")
    clf()

    println(convergence_orders(errors["N"], errors["||φ - φ_ana||_M"]))
    plot(errors["N"], errors["||φ - φ_ana||_M"], label=L"||\varphi - \varphi_{\mathrm{ana}}||_M", 
        marker=markers[1])

    println(convergence_orders(errors["N"], errors["||v - v_ana||_M"]))
    plot(errors["N"], errors["||v - v_ana||_M"], label=L"||v - v_{\mathrm{ana}}||_M", 
        marker=markers[2])

    println(convergence_orders(errors["N"], errors["||grad φ - u_irr||_M"]))
    plot(errors["N"], errors["||grad φ - u_irr||_M"], label=L"||\mathrm{grad}\, \varphi - u_{\mathrm{irr}}||_M", 
        marker=markers[3])

    println(convergence_orders(errors["N"], errors["||curl v - u_sol||_M"]))
    plot(errors["N"], errors["||curl v - u_sol||_M"], label=L"||\mathrm{curl}\, v - u_{\mathrm{sol}}||_M", 
        marker=markers[4])

    println(convergence_orders(errors["N"], errors["||r||_M"]))
    plot(errors["N"], errors["||r||_M"], label=L"||r||_M", 
        marker=markers[5])
    
    order_color = "silver"
    if order == 2
        plot([41, 121], [1.5e-3, 1.5e-3*(41/121)^2], "-", color=order_color)
        annotate("Order 2", (41, 7.e-4), color=order_color)
    elseif order == 4
        plot([41, 121], [1.5e-4, 1.5e-4*(41/121)^3.6], "-", color=order_color)
        annotate("Order 3.6", (41, 3.5e-5), color=order_color)
        plot([41, 121], [2.e-3, 2.e-3*(41/121)^3], "-", color=order_color)
        annotate("Order 3", (45, 1.6e-3), color=order_color)
    elseif order == 6
        plot([41, 121], [6.e-5, 6.e-5*(41/121)^4.6], "-", color=order_color)
        annotate("Order 4.6", (41, 1.e-5), color=order_color)
        plot([41, 121], [2.e-3, 2.e-3*(41/121)^4], "-", color=order_color)
        annotate("Order 4", (45, 1.5e-3), color=order_color)
    elseif order == 8
        plot([41, 121], [1.5e-5, 1.5e-5*(41/121)^5.7], "-", color=order_color)
        annotate("Order 5.7", (41, 1.5e-6), color=order_color)
        plot([41, 121], [2.e-4, 2.e-4*(41/121)^5], "-", color=order_color)
        annotate("Order 5", (45, 1.5e-4), color=order_color)
    end

    xlabel(L"N")
    xscale("log")
    ylabel("Error")
    yscale("log")
    savefig("../figures/convergence_3D__order_$order.pdf", bbox_inches="tight")
    
    # create a legend
    order == 2 || continue
    ax = gca()
    handles, labels = ax.get_legend_handles_labels()
    figure()
    figlegend(handles, labels, loc="center", ncol=5)
    savefig("../figures/convergence_3D__legend.pdf", bbox_inches="tight")
end


# Example: MHD Modes

In the following, the application to the detection of MHD wave modes are presented.

In [ ]:
function decompose_current_density(N1, N2, N3, order, k1, k3, εa, εf;
                                    scalar_potential_first=true,
                                    atol=1.e-12, btol=1.e-12, linsolve=lsmr)
    
    # background magentic field
    b1_background = (x1,x2,x3) -> zero(x1+x2+x3)
    b2_background = (x1,x2,x3) -> zero(x1+x2+x3)
    b3_background = (x1,x2,x3) -> one(x1+x2+x3)

    # Alfvén wave
    b1_alfven = (x1,x2,x3) -> εa * zero(x1+x2+x3)
    b2_alfven = (x1,x2,x3) -> εa * sin(k1 * x1 + k3 * x3)
    b3_alfven = (x1,x2,x3) -> εa * zero(x1+x2+x3)

    # fast magnetosonic wave
    b1_fastmode = (x1,x2,x3) -> εf * zero(x1+x2+x3)
    b2_fastmode = (x1,x2,x3) -> εf * zero(x1+x2+x3)
    b3_fastmode = (x1,x2,x3) -> -εf * sin(k1*x1 + k3*x3)

    # complete magnetic field
    b1func = (x1,x2,x3) -> b1_background(x1,x2,x3) + b1_alfven(x1,x2,x3) + b1_fastmode(x1,x2,x3)
    b2func = (x1,x2,x3) -> b2_background(x1,x2,x3) + b2_alfven(x1,x2,x3) + b2_fastmode(x1,x2,x3)
    b3func = (x1,x2,x3) -> b3_background(x1,x2,x3) + b3_alfven(x1,x2,x3) + b3_fastmode(x1,x2,x3)
    
    # compute the current density in the z=0 plane
    x1_3D, x2_3D, x3_3D, _, curl_3D, _, _, _ = operators(N1, N2, N3, order)
    b1 = b1func.(x1_3D, x2_3D, x3_3D)
    b2 = b2func.(x1_3D, x2_3D, x3_3D)
    b3 = b3func.(x1_3D, x2_3D, x3_3D)
    b = vcat(b1, b2, b3)
    
    j = curl_3D * b
    j1 = view(j, 1:N1*N2*N3)
    j2 = view(j, N1*N2*N3+1:2*N1*N2*N3)
    j01 = reshape(j1, N3, N2, N1)[N3÷2+1,:,:] |> vec
    j02 = reshape(j2, N3, N2, N1)[N3÷2+1,:,:] |> vec
    j0 = vcat(j01, j02)
    
    # project the current density in the z=0 plane
    x1, x2, grad, rot, curl, div, M, Mvec = operators(N1, N2, order)
    sqrtM = sqrt(M)
    sqrtMvec = sqrt(Mvec)
    
    retval = Dict{String, Any}()
    retval["||j^0||_M"] = sqrt(j0' * M * j0)
    
    if scalar_potential_first
        φ = sqrtM \ linsolve(sqrtMvec*grad/sqrtM, sqrtMvec*j0, atol=atol, btol=btol)
        gradφ = grad * φ
        r = j0 - gradφ
        retval["||j^0 - grad φ||_M"] = sqrt(r' * M * r)

        v = sqrtM \ linsolve(sqrtMvec*rot/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        rotv = rot * v
        @. r = r - rotv
        retval["||j^0 - grad φ - rot v||_M"] = sqrt(r' * M * r)
        retval["||r||_M"] = retval["||j^0 - grad φ - rot v||_M"]
    else
        v = sqrtM \ linsolve(sqrtMvec*rot/sqrtM, sqrtMvec*j0, atol=atol, btol=btol)
        rotv = rot * v
        r = j0 - rotv
        retval["||j^0 - rot v||_M"] = sqrt(r' * M * r)
        
        φ = sqrtM \ linsolve(sqrtMvec*grad/sqrtM, sqrtMvec*r, atol=atol, btol=btol)
        gradφ = grad * φ
        @. r = r - gradφ
        retval["||j^0 - rot v - grad φ||_M"] = sqrt(r' * M * r)
        retval["||r||_M"] = retval["||j^0 - rot v - grad φ||_M"]
    end
    
    retval["j^0"] = j0
    retval["grad φ"] = gradφ
    retval["rot v"] = rotv
    retval["r"] = r
    retval["x1"] = x1
    retval["x2"] = x2
    
    # global error
    diff = j01 - gradφ[1:N1*N2]
    retval["error of grad φ"] = sqrt(diff' * M * diff + gradφ[N1*N2+1:end]' * M * gradφ[N1*N2+1:end]) / sqrt(j01' * M * j01)
    @. diff = j02 - rotv[N1*N2+1:end]
    retval["error of rot v"] = sqrt(diff' * M * diff + rotv[1:N1*N2]' * M * rotv[1:N1*N2]) / sqrt(j02' * M * j02)
    
    # error in the interior
    j01_tmp    = reshape(j01,                N2, N1)[N2÷4:3N2÷4, N1÷4:3N1÷4] |> vec
    gradφ1_tmp = reshape(gradφ[1:N1*N2],     N2, N1)[N2÷4:3N2÷4, N1÷4:3N1÷4] |> vec
    gradφ2_tmp = reshape(gradφ[N1*N2+1:end], N2, N1)[N2÷4:3N2÷4, N1÷4:3N1÷4] |> vec
    j02_tmp    = reshape(j02,                N2, N1)[N2÷4:3N2÷4, N1÷4:3N1÷4] |> vec
    rotv1_tmp  = reshape(rotv[1:N1*N2],      N2, N1)[N2÷4:3N2÷4, N1÷4:3N1÷4] |> vec
    rotv2_tmp  = reshape(rotv[N1*N2+1:end],  N2, N1)[N2÷4:3N2÷4, N1÷4:3N1÷4] |> vec
    retval["interior error of grad φ"] = sqrt(sum(abs2, j01_tmp-gradφ1_tmp) + sum(abs2, gradφ2_tmp)) / sqrt(sum(abs2, j01_tmp))
    retval["interior error of rot v"] = sqrt(sum(abs2, j02_tmp-rotv2_tmp) + sum(abs2, rotv1_tmp)) / sqrt(sum(abs2, j02_tmp))
    
    retval
end

function my_colorbar(minmax)
    cb = colorbar()
    cb.formatter.set_powerlimits((0, 0))
    cb.update_ticks()
    minmax !== nothing && clim(-minmax, minmax)
    cb
end

function plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; 
                            cmax1=nothing, cmax2=cmax1, kwargs...)
    retval = decompose_current_density(N1, N2, N3, order, k1, k3, εa, εf; kwargs...)

    close("all")
    fig = figure(figsize=(12,7))
    clf()

    subplot(231)
    pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
        reshape(retval["j^0"][1:N1*N2], N2, N1), linewidth=0, rasterized=true)
    title(L"j^\perp_1")
    xlabel(L"x")
    ylabel(L"y")
    gca().set_aspect("equal")
    my_colorbar(cmax1)

    subplot(234)
    pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
        reshape(retval["j^0"][N1*N2+1:2*N1*N2], N2, N1), linewidth=0, rasterized=true)
    title(L"j^\perp_2")
    xlabel(L"x")
    ylabel(L"y")
    gca().set_aspect("equal")
    my_colorbar(cmax2)

    subplot(232)
    pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
        reshape(retval["grad φ"][1:N1*N2], N2, N1), linewidth=0, rasterized=true)
    title(L"(\mathrm{grad}\,\varphi)_1")
    xlabel(L"x")
    ylabel(L"y")
    gca().set_aspect("equal")
    my_colorbar(cmax1)

    subplot(235)
    pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
        reshape(retval["grad φ"][N1*N2+1:2*N1*N2], N2, N1), linewidth=0, rasterized=true)
    title(L"(\mathrm{grad}\,\varphi)_2")
    xlabel(L"x")
    ylabel(L"y")
    gca().set_aspect("equal")
    my_colorbar(cmax2)

    subplot(233)
    pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
        reshape(retval["rot v"][1:N1*N2], N2, N1), linewidth=0, rasterized=true)
    title(L"(\mathrm{rot}\,v)_1")
    xlabel(L"x")
    ylabel(L"y")
    gca().set_aspect("equal")
    my_colorbar(cmax1)

    subplot(236)
    pcolormesh(reshape(retval["x1"], N2, N1), reshape(retval["x2"], N2, N1), 
        reshape(retval["rot v"][N1*N2+1:2*N1*N2], N2, N1), linewidth=0, rasterized=true)
    title(L"(\mathrm{rot}\,v)_2")
    xlabel(L"x")
    ylabel(L"y")
    gca().set_aspect("equal")
    my_colorbar(cmax2)

    tight_layout()
    fig
end


### Experiments in 3D (Not Used in the Paper)

This cell provides the possibility to experiment a bit with different data and paramters.

In [ ]:
N1 = N2 = N3 = 101
order = 6
k1 = k3 = 5π
εa = 1.e-2
εf = 1.e-5

fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=true, cmax1=nothing);
#fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=true, cmax1=1.5e-1, cmax2=1.5e-4);

### Error Plot Used in the Paper

In [ ]:
N1 = N2 = N3 = 101
order = 6
k_factors = 1:5
εa = 1.e-3
εf = 1.e-2

err_rotv = zeros(length(k_factors))
err_gradφ = zeros(length(k_factors))
int_err_rotv = zeros(length(k_factors))
int_err_gradφ = zeros(length(k_factors))
for (idx,k_factor) in enumerate(k_factors)
    k1 = k3 = k_factor * π
    retval = decompose_current_density(N1, N2, N3, order, k1, k3, εa, εf, scalar_potential_first=false)
    err_rotv[idx] = retval["error of rot v"]
    err_gradφ[idx] = retval["error of grad φ"]
    int_err_rotv[idx] = retval["interior error of rot v"]
    int_err_gradφ[idx] = retval["interior error of grad φ"]
end
display(hcat(k_factors, err_rotv, int_err_rotv, err_gradφ, int_err_gradφ))

plt.clf()
plt.scatter(k_factors, err_rotv, label="Alfvén mode (\$ j^\\perp_1 \$ ), global", marker=markers[1])
plt.scatter(k_factors, int_err_rotv, label="Alfvén mode (\$ j^\\perp_1 \$ ), interior", marker=markers[2])
plt.scatter(k_factors, err_gradφ, label="Magnetosonic mode (\$ j^\\perp_2 \$ ), global", marker=markers[3])
plt.scatter(k_factors, int_err_gradφ, label="Magnetosonic mode (\$ j^\\perp_2 \$ ), interior", marker=markers[4])
xlabel(L"k / \pi")
ylabel("Relative Error")
yscale("log")
ylim(1.e-5, 1.e0)
legend(loc="upper left", bbox_to_anchor=(1,1))
savefig("../figures/MHD_modes__error_vs_k.pdf", bbox_inches="tight")

### Plots Used in the Paper

In [ ]:
N1 = N2 = N3 = 81
order = 6
k1 = π; k3 = π
εa = 1.e-2; εf = 1.e-2
#fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=true, cmax1=3.e-2);
#fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_1em2__ef_1em2__scalar.pdf", bbox_inches="tight");
#fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=false, cmax1=3.e-2);
#fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_1em2__ef_1em2__vector.pdf", bbox_inches="tight");

N1 = N2 = N3 = 81
order = 6
k1 = π; k3 = π
εa = 0.; εf = 1.e-2
#fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=true, cmax1=3.e-2);
#fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_0__ef_1em2__scalar.pdf", bbox_inches="tight");
#fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=false, cmax1=nothing);
#fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_0__ef_1em2__vector.pdf", bbox_inches="tight");

N1 = N2 = N3 = 81
order = 6
k1 = π; k3 = π
εa = 1.e-2; εf = 0.
#fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=true, cmax1=nothing);
#fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_1em2__ef_0__scalar.pdf", bbox_inches="tight");
#fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=false, cmax1=3.e-2);
#fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_1em2__ef_0__vector.pdf", bbox_inches="tight");

N1 = N2 = N3 = 101
order = 6
k1 = k3 = 5π
εa = 1.e-2; εf = 1.e-5
fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=true, cmax1=1.5e-1, cmax2=1.5e-4);
fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_1em2__ef_0__scalar.pdf", bbox_inches="tight");
fig = plot_and_decompose(N1, N2, N3, order, k1, k3, εa, εf; scalar_potential_first=false, cmax1=1.5e-1, cmax2=nothing);
fig.savefig("../figures/MHD_modes__k1_pi__k3_pi__ea_1em2__ef_0__vector.pdf", bbox_inches="tight");